Date Created - 13th September 2020

Change log - Updating the code to
1. Catch the event data when available
2. Add the patients at risk data


## Basics

In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
import json
from pandas.io.json import json_normalize
import xlrd

In [2]:
import plotly.express as px 
import plotly.graph_objects as go

import dash  
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table_experiments as dt

In [26]:
import dash_table

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
nctid = 'NCT01859988'

## AE Fetching Functions

In [5]:
def decode_event_group(coded_value, event_group):
    '''
    Decode arm names, example - EG000 to 'Dupilumab 300 mg qw'
    '''
    decoded_value = event_group[event_group.EventGroupId==coded_value]['EventGroupTitle'].values[0]
    if not decoded_value:
        decoded_value = coded_value
    return decoded_value

def get_oae(nctid):
    # Get CT.gov data on the NCTID
    URL = f'https://clinicaltrials.gov/api/query/full_studies?expr={nctid}&max_rnk=1&fmt=JSON'
    r = requests.get(URL)
    j = json.loads(r.content)
    # Other AE data
    tt = j['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['AdverseEventsModule']['OtherEventList']['OtherEvent']
    event_groups = pd.json_normalize(j['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['AdverseEventsModule']['EventGroupList']['EventGroup'])
    # convert into tabular format
    tt2 = pd.json_normalize(tt,
              ['OtherEventStatsList','OtherEventStats'],
              ['OtherEventTerm', 'OtherEventOrganSystem'],
              errors='ignore')
    # convert into multi-indexed column
    #if tt[0]['OtherEventStatsList']['OtherEventStats'][0]['OtherEventStatsNumEvents']:
    try:
        tt3 = tt2.pivot(columns='OtherEventStatsGroupId',
        values=['OtherEventStatsNumAffected','OtherEventStatsNumEvents','OtherEventStatsNumAtRisk'],
        index='OtherEventTerm')
        tt3.rename(columns={'OtherEventStatsNumEvents':'Events'}, inplace=True, level=0)
    except KeyError:
        tt3 = tt2.pivot(columns='OtherEventStatsGroupId',
        values=['OtherEventStatsNumAffected','OtherEventStatsNumAtRisk'],
        index='OtherEventTerm')
    tt3.rename(columns=lambda x: decode_event_group(x,event_groups), inplace=True, level=1)
    tt3.rename(columns={'OtherEventStatsNumAffected':'Subjects','OtherEventStatsNumAtRisk':'Total_Subjects'}, inplace=True, level=0)
    return(tt3)

def get_sae(nctid):
    # Get CT.gov data on the NCTID
    URL = f'https://clinicaltrials.gov/api/query/full_studies?expr={nctid}&max_rnk=1&fmt=JSON'
    r = requests.get(URL)
    j = json.loads(r.content)
    # Other AE data
    tt = j['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['AdverseEventsModule']['SeriousEventList']['SeriousEvent']
    event_groups = pd.json_normalize(j['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['AdverseEventsModule']['EventGroupList']['EventGroup'])
    # convert into tabular format
    tt2 = pd.json_normalize(tt,
              ['SeriousEventStatsList','SeriousEventStats'],
              ['SeriousEventTerm', 'SeriousEventOrganSystem'],
              errors='ignore')
    # convert into multi-indexed column
    try:
        tt3 = tt2.pivot(columns='SeriousEventStatsGroupId',
                    values=['SeriousEventStatsNumAffected','SeriousEventStatsNumEvents','SeriousEventStatsNumAtRisk'],
                    index='SeriousEventTerm')
        tt3.rename(columns={'SeriousEventStatsNumEvents':'Events'}, inplace=True, level=0)
    except KeyError:
        tt3 = tt2.pivot(columns='SeriousEventStatsGroupId',
                    values=['SeriousEventStatsNumAffected','SeriousEventStatsNumAtRisk'],
                    index='SeriousEventTerm')
    tt3.rename(columns=lambda x: decode_event_group(x,event_groups), inplace=True, level=1)
    tt3.rename(columns={'SeriousEventStatsNumAffected':'Subjects','SeriousEventStatsNumAtRisk':'Total_Subjects'}, inplace=True, level=0)
    return(tt3)

In [18]:
#event_groups = pd.json_normalize(j['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['AdverseEventsModule']['EventGroupList']['EventGroup'])
#event_groups.iloc[:,:3]

In [19]:
#decode_event_group(coded_value = 'EG000', event_group=event_groups)

## Testing

### Test 1

In [20]:
nctid = 'NCT01859988'

In [21]:
get_sae(nctid)

Subjects                                                                                                          Events                                                                                                  Total_Subjects                                                                                            
SeriousEventStatsGroupId Dupilumab 300 mg qw Dupilumab 300 mg q2w Dupilumab 200 mg q2w Dupilumab 300 mg q4w Dupilumab 100 mg q4w Placebo Dupilumab 300 mg qw Dupilumab 300 mg q2w Dupilumab 200 mg q2w Dupilumab 300 mg q4w Dupilumab 100 mg q4w Placebo Dupilumab 300 mg qw Dupilumab 300 mg q2w Dupilumab 200 mg q2w Dupilumab 300 mg q4w Dupilumab 100 mg q4w Placebo
SeriousEventTerm                                                                                                                                                                                                                                                                                                                                                        
Abortion induced                           0                    0                    0                    0                    0       1                   0                    0                    0                    0                    0       1                  63                   64                   61                   65                   65      61
Anaphylactic shock                         0                    0                    1                    0                    0       0                   0                    0                    1                    0                    0       0                  63                   64                   61                   65                   65      61
Asthma                                     0                    0                    0                    0                    1       0                   0                    0                    0                    0                    1       0                  63                   64                   61                   65                   65      61
Cellulitis                                 0                    0                    0                    0                    1       0                   0                    0                    0                    0                    1       0                  63                   64                   61                   65                   65      61
Dermatitis atopic                          0                    1                    0                    0                    4       1                   0                    1                    0                    0                    5       1                  63                   64                   61                   65                   65      61
Dermatitis exfoliative                     0                    0                    0                    1                    0       0                   0                    0                    0                    1                    0       0                  63                   64                   61                   65                   65      61
Hip dysplasia                              0                    0                    0                    0                    0       1                   0                    0                    0                    0                    0       1                  63                   64                   61                   65                   65      61
Osteonecrosis                              0                    0                    0                    0                    0       1                   0                    0                    0                    0                    0       1                  63                   64                   61                   65                   65      61
Peritonsillar abscess                      0                    0

In [16]:
tt = get_oae(nctid)
tt

Subjects                                                                                                          Events                                                                                                  Total_Subjects                                                                                            
OtherEventStatsGroupId            Dupilumab 300 mg qw Dupilumab 300 mg q2w Dupilumab 200 mg q2w Dupilumab 300 mg q4w Dupilumab 100 mg q4w Placebo Dupilumab 300 mg qw Dupilumab 300 mg q2w Dupilumab 200 mg q2w Dupilumab 300 mg q4w Dupilumab 100 mg q4w Placebo Dupilumab 300 mg qw Dupilumab 300 mg q2w Dupilumab 200 mg q2w Dupilumab 300 mg q4w Dupilumab 100 mg q4w Placebo
OtherEventTerm                                                                                                                                                                                                                                                                                                                                                                   
Abdominal pain upper                                1                    2                    0                    0                    4       1                   1                    2                    0                    0                    4       1                  63                   64                   61                   65                   65      61
Arthralgia                                          1                    4                    4                    1                    1       0                   1                    4                    5                    1                    1       0                  63                   64                   61                   65                   65      61
Back pain                                           2                    2                    0                    2                    3       5                  16                    3                    0                    5                    3       5                  63                   64                   61                   65                   65      61
Blood triglycerides increased                       0                    1                    0                    4                    0       0                   0                    1                    0                    5                    0       0                  63                   64                   61                   65                   65      61
Conjunctivitis                                      4                    1                    0                    1                    0       0                   5                    1                    0                    1                    0       0                  63                   64                   61                   65                   65      61
Conjunctivitis allergic                             3                    2                    6                    3                    1       2                   5                    4                    9                    3                    1       2                  63                   64                   61                   65                   65      61
Cough                                               4                    4                    2                    1                    0       1                   4                    4                    2                    1                    0       1                  63                   64                   61                   65                   65      61
Dermatitis atopic                                   8                   13                    8                   10                   11      10                   9                   19                   10                   12                   13      12                  63                   64                   61                   65     

In [10]:
tt['Subjects']

OtherEventStatsGroupId,Dupilumab 300 mg qw,Dupilumab 300 mg q2w,Dupilumab 200 mg q2w,Dupilumab 300 mg q4w,Dupilumab 100 mg q4w,Placebo
OtherEventTerm,,,,,,
Abdominal pain upper,1,2,0,0,4,1
Arthralgia,1,4,4,1,1,0
Back pain,2,2,0,2,3,5
Blood triglycerides increased,0,1,0,4,0,0
Conjunctivitis,4,1,0,1,0,0
Conjunctivitis allergic,3,2,6,3,1,2
Cough,4,4,2,1,0,1
Dermatitis atopic,8,13,8,10,11,10
Fatigue,2,1,1,4,0,3


### Test 2

In [9]:
nctid = 'NCT02369484'

In [10]:
get_oae(nctid)

,Subjects,OtherEventStatsNumAtRisk
OtherEventStatsGroupId,Afatinib,Afatinib
OtherEventTerm,,
Abdominal pain,3,13
Alopecia,1,13
Anemia,2,13
Arthalgia,1,13
Asparate aminotransferase increased,1,13
Back pain,1,13
Bladder infection,1,13
Bone pain,1,13


In [13]:
get_sae(nctid)

,Subjects
SeriousEventStatsGroupId,Afatinib
SeriousEventTerm,
Acute kidney injury,1
Dehydration,1
Diarrhea,1
Dyspnea,1
Epistaxis,1
Febrile neutropenia,1
Muscle weakness lower limb,1
Pericardial effusion,1


### Test 3

In [14]:
nctid = 'NCT00770588'

In [15]:
get_oae(nctid)

Subjects        
OtherEventStatsGroupId               Gefitinib Placebo
OtherEventTerm                                        
Alanine Aminotransferase Increased          31      12
aspartate aminotransferase increased        21       6
cough                                        9      20
diarrhoea                                   37      13
dry skin                                     9       3
pruritus                                    10       7
rash                                        73      14
serious hepatic dysfunction                 43      16
skin exfoliation                             9       0
transaminases increased                      8       2

In [16]:
get_sae(nctid)

Subjects        
SeriousEventStatsGroupId             Gefitinib Placebo
SeriousEventTerm                                      
Accidental Death                             0       1
Alanine Aminotransferase Increased           0       1
Arterial Thrombosis Limb                     1       0
Aspartate Aminotransferase Increased         0       1
Circulatory Collapse                         1       0
Completed Suicide                            0       1
Death                                        1       0
Haemoptysis                                  1       0
Interstitial Lung Disease                    2       0
Lung Infection                               2       0
Paraplegia                                   1       0
Pleural Effusion                             0       1
Pneumonia                                    2       0
Subcutaneous Nodule                          0       1
Sudden Death                                 1       0

In [17]:
get_sae(nctid)

Subjects        
SeriousEventStatsGroupId             Gefitinib Placebo
SeriousEventTerm                                      
Accidental Death                             0       1
Alanine Aminotransferase Increased           0       1
Arterial Thrombosis Limb                     1       0
Aspartate Aminotransferase Increased         0       1
Circulatory Collapse                         1       0
Completed Suicide                            0       1
Death                                        1       0
Haemoptysis                                  1       0
Interstitial Lung Disease                    2       0
Lung Infection                               2       0
Paraplegia                                   1       0
Pleural Effusion                             0       1
Pneumonia                                    2       0
Subcutaneous Nodule                          0       1
Sudden Death                                 1       0

### Test 4

In [18]:
nctid = 'NCT00820755'

In [19]:
get_oae(nctid)

Subjects                                                               
OtherEventStatsGroupId             Cetuximab 250 mg/m^2 q1w + Platinum-based Doublet Chemotherapy Cetuximab 500 mg/m^2 Every 2 Weeks Cetuximab 250 mg/m^2 Weekly
OtherEventTerm                                                                                                                                                  
Abdominal pain                                                                     21                                              0                           0
Abdominal pain upper                                                               31                                              0                           0
Acne                                                                               31                                              0                           0
Alanine aminotransferase increased                                                 26                                              0                           0
Alopecia                                                                          109                                              0                           0
Anaemia                                                                           134                                              9                          15
Arthralgia                                                                         33                                              5                          11
Asthenia                                                                          124                                              6                          16
Back pain                                                                          30                                              6                           8
Bone pain                                                                           0                                              2                           8
Chills                                                                             16                                              0                           0
Conjunctivitis                                                                      0                                              7                          10
Constipation                                                                      109                                              3                          12
Cough                                                                              43                                             20                          22
Decreased appetite                                                                112                                              8                          13
Dermatitis acneiform                                                               83                                             11                          16
Diarrhoea                                                                         120                                              6                          15
Dizziness                                                                          35                                              4                           8
Dry skin                                                                           46                                              9                          11
Dysgeusia                                                                          28                                              0                           0
Dyspepsia                                                                          36                                              0                           0
Dysphonia                                                                          17                                              0                           0
Dyspnoea                                                        

In [20]:
get_sae(nctid)

Subjects                                                               
SeriousEventStatsGroupId                    Cetuximab 250 mg/m^2 q1w + Platinum-based Doublet Chemotherapy Cetuximab 500 mg/m^2 Every 2 Weeks Cetuximab 250 mg/m^2 Weekly
SeriousEventTerm                                                                                                                                                         
Abdominal distension                                                                         1                                              0                           0
Abdominal pain                                                                               2                                              0                           0
Acute myocardial infarction                                                                  1                                              0                           0
Acute respiratory failure                                                                    2                                              0                           0
Adrenal insufficiency                                                                        0                                              0                           1
Alanine aminotransferase increased                                                           1                                              0                           1
Altered state of consciousness                                                               0                                              1                           0
Anaemia                                                                                      6                                              1                           1
Anaphylactic reaction                                                                        3                                              0                           0
Anaphylactic shock                                                                           1                                              0                           0
Anastomotic ulcer                                                                            1                                              0                           0
Angina unstable                                                                              1                                              0                           0
Anorectal infection                                                                          1                                              0                           0
Aortic thrombosis                                                                            1                                              0                           0
Aplasia                                                                                      1                                              0                           0
Arrhythmia superaventricular                                                                 1                                              0                           0
Arterial thrombosis                                                                          0                                              0                           1
Arthralgia                                                                                   1                                              0                           0
Asthenia                                                                                     8                                              1                           0
Atrial fibrillation                                                                          3                                              0                           0
Back pain                                                                                    5                                              0                           0
Blood creatinine i

## Dash App

In [35]:
app = dash.Dash(__name__)

In [36]:
# App layout
app.layout = html.Div([

    html.H1("Compare AE Results from Clinical Trials", style={'text-align': 'center'}),

    html.Div([
        html.Br(),
        dcc.Input(id="trial1", type="text", placeholder="Trial Id of the first trial"),
        html.Br(),
        dcc.Input(id="trial2", type="text", placeholder="Trial Id of the second trial", debounce=True),
    ]),
    
    html.Div(id='output-data-upload'),
    html.Br(),

    #dcc.Graph(id='trial_en_graph', figure={})

])


# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    dash.dependencies.Output('output-data-upload', 'children'),
     #Output(component_id='output_container_slide', component_property='children')],
    [Input(component_id='trial1', component_property='value'),
     Input(component_id='trial2', component_property='value')]
     #Input(component_id='range_slider', component_property='value')]
)

# Business logic
def update_graph(trial1, trial2):
    print(trial1)
    print(trial2)
    
    df = get_sae(trial1)
    df = df['Subjects']
    df.reset_index(inplace=True)
    print(df)
    #df['AdverseEvent'] = df.index() 
    return html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict("rows"),
        style_cell={'width': '300px',
        'height': '60px',
        'textAlign': 'left'})
    ])


# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Sep/2020 21:37:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2020 21:37:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2020 21:37:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2020 21:37:42] "GET /_favicon.ico?v=1.16.0 HTTP/1.1" 200 -


None
None
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rul

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint

127.0.0.1 - - [13/Sep/2020 21:37:44] "POST /_dash-update-component HTTP/1.1" 500 -


NCT01859988
None


/home/ubuntu/anaconda3/envs/fastai2/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

127.0.0.1 - - [13/Sep/2020 21:37:45] "POST /_dash-update-component HTTP/1.1" 200 -


SeriousEventStatsGroupId        SeriousEventTerm Dupilumab 300 mg qw Dupilumab 300 mg q2w Dupilumab 200 mg q2w Dupilumab 300 mg q4w Dupilumab 100 mg q4w Placebo
0                               Abortion induced                   0                    0                    0                    0                    0       1
1                             Anaphylactic shock                   0                    0                    1                    0                    0       0
2                                         Asthma                   0                    0                    0                    0                    1       0
3                                     Cellulitis                   0                    0                    0                    0                    1       0
4                              Dermatitis atopic                   0                    1                    0                    0                    4       1
5                         Dermatit